In [ ]:
# Run this cell to open ngrok tunnel that will give colab session access from application.
# Registration at https://ngrok.com/ required to get Authorization token.
# You will be prompted to paste your ngrok authtoken in this cell output field.
# After successful execution of script - connection credentials will be saved in 
# /content/ssh_config/credentials file.
import getpass
import requests
import secrets
import time

from requests.adapters import HTTPAdapter, Retry

print("Configure SSH connection in progress...")

# Install and configure open-ssh server
! apt update > /dev/null 2>/dev/null && apt install openssh-server > /dev/null 2>/dev/null
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
get_ipython().system_raw('service ssh start')

# Create user credentials with randomly generated password
password = secrets.token_urlsafe(32)
! mkdir -p ./ssh_config/
! echo user=$(whoami) > ./ssh_config/credentials
! echo password=$password >> ./ssh_config/credentials
! echo $(whoami):$password | chpasswd

# Install and configure ngrok for connection tunelling with provided authtoken
! wget -q -c -nc https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz && \
  tar xvzf ngrok-v3-stable-linux-amd64.tgz -C /usr/local/bin > /dev/null && \
  rm ngrok-v3-stable-linux-amd64.tgz
authtoken = getpass.getpass(
  prompt="Paste authtoken from https://dashboard.ngrok.com/auth here:\n"
)
get_ipython().system_raw("ngrok authtoken $authtoken && ngrok tcp 22 &")

# Add ngrok tunnel host and port to credentials file
session = requests.Session()
retries = Retry(total=3, backoff_factor=1)
session.mount('http://', HTTPAdapter(max_retries=retries))
response = session.get("http://localhost:4040/api/tunnels", timeout=30)
host_data = response.json()
host, port = host_data["tunnels"][0]["public_url"][6:].split(":")

! echo host=$host >> ./ssh_config/credentials
! echo port=$port >> ./ssh_config/credentials

print("Succeed. Credentials file created at /content/ssh_config/credentials")